In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import pickle
import numpy as np
import os
from ExpPipeline import loadResults, plotAvgWithError, plotCDF
plt.ioff()

In [2]:
def plotSyntheticExp(expDir: str):
    expName = os.path.basename(expDir)
    resultPath = os.path.join(expDir, "evalResults.pkl")
    (
        baseObjList, 
        MoVEObjList, 
        ROVEObjList, 
        ROVEsObjList, 
        baseObjAvg, 
        MoVEObjAvg, 
        ROVEObjAvg, 
        ROVEsObjAvg,
        sampleSizeList, 
        kList, 
        BList, 
        k12List, 
        B12List, 
        numReplicates,
    ) = loadResults(resultPath)
    baggingObjAvg = []
    baggingObjList = []
    
    # if expDir.endswith("NN_d30_l4_ES_WB"):
    if False:
        fileList = [
            "1024_2048.pkl",
            "4096_8192.pkl",
            "16384_32768.pkl",
            "65536.pkl",
        ]
        data = {}
        for file in fileList:
            with open(os.path.join(expDir, file), "rb") as f:
                data.update(pickle.load(f))
        
        for size in sorted(data):
            valueDict = data[size]
            baggingObjList.append([valueDict[idx] for idx in sorted(valueDict)])
            baggingObjAvg.append(np.mean(baggingObjList[-1]))

    if len(baggingObjAvg) > 0 and len(baggingObjAvg) != len(sampleSizeList):
        raise ValueError("incorrect size for bagging objective avg list")
    if len(baggingObjList) > 0 and len(baggingObjList) != len(sampleSizeList):
        raise ValueError("incorrect size for bagging objective list")
    plotAvgWithError(
        baseObjList,
        MoVEObjList,
        ROVEObjList,
        ROVEsObjList,
        baggingObjList,
        numReplicates,
        0.95,
        sampleSizeList,
        os.path.join(expDir, f"{expName}_plots/{expName}_avg.png"),
        yLogScale=False
    )
    plotAvgWithError(
        baseObjList,
        MoVEObjList,
        ROVEObjList,
        ROVEsObjList,
        baggingObjList,
        numReplicates,
        0.95,
        sampleSizeList,
        os.path.join(expDir, f"{expName}_plots/{expName}_avg_ylog.png"),
        yLogScale=True
    )

    # if expDir.endswith("NN_d30_l4_ES_WB"):
    for i in range(len(sampleSizeList)):
        plotCDF(
            baseObjList[i],
            MoVEObjList[i],
            ROVEObjList[i],
            ROVEsObjList[i],
            baggingObjList[i] if len(baggingObjList) > 0 else [],
            os.path.join(expDir, f"{expName}_plots/{expName}_cdf_{int(np.log2(sampleSizeList[i]))}.png"),
            xLogScale = False,
            yLogScale = True
        )
        plotCDF(
            baseObjList[i],
            MoVEObjList[i],
            ROVEObjList[i],
            ROVEsObjList[i],
            baggingObjList[i] if len(baggingObjList) > 0 else [],
            os.path.join(expDir, f"{expName}_plots/{expName}_cdf_{int(np.log2(sampleSizeList[i]))}_xlog.png"),
            xLogScale = True,
            yLogScale = True
        )

In [3]:
def plotRealExp(expDir: str):
    expName = os.path.basename(expDir)
    resultPath = os.path.join(expDir, "evalResults.pkl")
    (
        baseObjList, 
        MoVEObjList, 
        ROVEObjList, 
        ROVEsObjList, 
        baseObjAvg, 
        MoVEObjAvg, 
        ROVEObjAvg, 
        ROVEsObjAvg,
        sampleSizeList, 
        kList, 
        BList, 
        k12List, 
        B12List, 
        numReplicates,
    ) = loadResults(resultPath)
    baggingObjList = []
    
    plotCDF(
        baseObjList[0],
        MoVEObjList[0],
        ROVEObjList[0],
        ROVEsObjList[0],
        baggingObjList[0] if len(baggingObjList) > 0 else [],
        os.path.join(expDir, f"{expName}_plots/{expName}_cdf.png"),
        xLogScale = False,
        yLogScale = True
    )
    plotCDF(
        baseObjList[0],
        MoVEObjList[0],
        ROVEObjList[0],
        ROVEsObjList[0],
        baggingObjList[0] if len(baggingObjList) > 0 else [],
        os.path.join(expDir, f"{expName}_plots/{expName}_cdf_xlog.png"),
        xLogScale = True,
        yLogScale = True
    )

# Plot Exp Results

In [125]:
masterDir = "/home/hqian/ResearchProjects/BaggingSAA/ExpData/NNData/RealData"
expDirList = os.listdir(masterDir)
for expDir in expDirList:
    plotRealExp(os.path.join(masterDir, expDir))

In [6]:
masterDir = "/home/hqian/ResearchProjects/BaggingSAA/ExpData/NNData/SyntheticData"
expDirList = os.listdir(masterDir)
for expDir in expDirList:
    plotSyntheticExp(os.path.join(masterDir, expDir))

In [7]:
masterDir = "/home/hqian/ResearchProjects/BaggingSAA/ExpData/LRData"
expDirList = os.listdir(masterDir)
for expDir in expDirList:
    plotSyntheticExp(os.path.join(masterDir, expDir))

/tmp/ipykernel_213129/4062217522.py:80: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()


In [8]:
plotSyntheticExp("/home/hqian/ResearchProjects/BaggingSAA/ExpData/Port_d10")
plotSyntheticExp("/home/hqian/ResearchProjects/BaggingSAA/ExpData/Port_d50")